# Unsupervised NMT

## Installation

In [1]:
# Mount Colab
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# Install pytorch using wheel
!pip3 install torch torchvision

    100% |████████████████████████████████| 519.5MB 33kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x590a4000 @  0x7fc42c3182a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
    100% |████████████████████████████████| 61kB 22.3MB/s 
    100% |████████████████████████████████| 2.0MB 4.5MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [3]:
# Install the github repo
!git clone https://github.com/jaderabbit/UnsupervisedMT.git


Cloning into 'UnsupervisedMT'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 122 (delta 32), reused 29 (delta 17), pack-reused 71
Receiving objects: 100% (122/122), 112.33 KiB | 2.44 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [11]:
#!cd UnsupervisedMT; git pull origin master

remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 8 (delta 6), reused 8 (delta 6), pack-reused 0
Unpacking objects: 100% (8/8), done.
From https://github.com/jaderabbit/UnsupervisedMT
 * branch            master     -> FETCH_HEAD
   c51738b..12112a4  master     -> origin/master
Updating c51738b..12112a4
Fast-forward
 NMT/src/utils.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


## English-Zulu NMT

### Generate Data

In [4]:
# cd into github repo
# Run preparation
!cd UnsupervisedMT/NMT; ./get_my_data.sh zu https://github.com/LauraMartinus/ukuxhumana/raw/master/leipzig/web_2013_100K_mono.zu


===== Input parameters
 source: en
 target: zu
 target monolingual url: https://github.com/LauraMartinus/ukuxhumana/raw/master/leipzig/web_2013_100K_mono.zu
 
Cloning Moses from GitHub repository...
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 147198, done.
remote: Total 147198 (delta 0), reused 0 (delta 0), pack-reused 147198
Receiving objects: 100% (147198/147198), 129.67 MiB | 18.82 MiB/s, done.
Resolving deltas: 100% (113765/113765), done.
Moses found in: /content/UnsupervisedMT/NMT/tools/mosesdecoder
Cloning fastBPE from GitHub repository...
Cloning into 'fastBPE'...
remote: Enumerating objects: 24, done.
remote: Total 24 (delta 0), reused 0 (delta 0), pack-reused 24
Unpacking objects: 100% (24/24), done.
fastBPE found in: /content/UnsupervisedMT/NMT/tools/fastBPE
Compiling fastBPE...
fastBPE compiled in: /content/UnsupervisedMT/NMT/tools/fastBPE/fast
Cloning fastText from GitHub repository...
Cloning into 'fastText'...
remote: Enumerating objects: 91, done.
remote

In [0]:
# If you want to save to drive
!cp -r /content/UnsupervisedMT/NMT/data/* "/content/gdrive/My Drive/NMT/data/"

# If you want to pull in from drive
#!cp -r "/content/gdrive/My Drive/NMT/data/*" /content/UnsupervisedMT/NMT/data/


In [0]:
# Save weights to drive
file_path='/content/UnsupervisedMT/NMT/data/mono/all.en-zu.60000.vec'
destination_path='/content/gdrive/My Drive/NMT/all.en-zu.60000.vec'
with open(destination_path, 'w') as f:
  with open(file_path,'r') as to_move:
    f.write(to_move.read())


In [12]:
# Run the unsupervised nmt
!cd UnsupervisedMT/NMT; python -u main.py --exp_name enzu     --transformer True  --n_enc_layers 4 --n_dec_layers 4      --share_enc 3  --share_dec 3  --share_lang_emb True  --share_output_emb True     --langs "en,zu" --n_mono -1  --mono_dataset "en:./data/mono/mono.en.tok.60000.pth,,;zu:./data/mono/mono.zu.tok.60000.pth,,"  --para_dataset "en-zu:,./data/para/enzu_parallel.dev.en.60000.pth,./data/para/enzu_parallel.dev.zu.60000.pth"     --mono_directions "en,zu"  --word_shuffle 3 --word_dropout 0.1 --word_blank 0.2     --pivo_directions "en-zu-en,zu-en-zu"     --pretrained_emb "/content/gdrive/My Drive/NMT/all.en-zu.60000.vec"  --pretrained_out True     --lambda_xe_mono '0:1,100000:0.1,300000:0' --lambda_xe_otfd 1     --otf_num_processes 30   --otf_sync_params_every 1000     --enc_optimizer adam,lr=0.0001     --group_by_size True     --batch_size 32     --epoch_size 500000 --stopping_criterion bleu_en_zu_valid,10     --freeze_enc_emb False --freeze_dec_emb False     --dump_path "/content/gdrive/My Drive/NMT"        


INFO - 11/21/18 19:53:14 - 0:00:00 - ============ Initialized logger ============
INFO - 11/21/18 19:53:14 - 0:00:00 - attention: True
                                     attention_dropout: 0
                                     back_dataset: {}
                                     back_directions: []
                                     batch_size: 32
                                     beam_size: 0
                                     clip_grad_norm: 5
                                     command: python main.py --exp_name 'enzu' --transformer 'True' --n_enc_layers '4' --n_dec_layers '4' --share_enc '3' --share_dec '3' --share_lang_emb 'True' --share_output_emb 'True' --langs 'en,zu' --n_mono '-1' --mono_dataset 'en:./data/mono/mono.en.tok.60000.pth,,;zu:./data/mono/mono.zu.tok.60000.pth,,' --para_dataset 'en-zu:,./data/para/enzu_parallel.dev.en.60000.pth,./data/para/enzu_parallel.dev.zu.60000.pth' --mono_directions 'en,zu' --word_shuffle '3' --word_dropout '0.1' --word_blank '0.2'